<a href="https://colab.research.google.com/github/kapoor1309/MathAgent/blob/main/MathAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install crewai
!pip install crewai_tools
!pip install 'crewai[tools]'

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
!pip install langchain-groq langchain --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 2.6 MB/s eta 0:00:00


In [ ]:
from crewai import Agent,Task,Crew,Process
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from crewai import LLM

load_dotenv()

llm=LLM(
    model='groq/gemma2-9b-it',
    temperature=0.7,
    api_key='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'
)



# #agents

# story_outliner=Agent(
#     role="story outliner",
#     goal='Develop an outline for a children storybook about animals including chapter titles and character names',
#     backstory='an imaginative creator who lays the foundation of captivating stories for children',

#     verbose=True,
#     llm=llm,
#     allow_delegation=False
# )

# story_writer=Agent(
#     role='Story Writer',
#     goal='write the full content of the story for all 5 chapters',
#     backstory='A talented wordsmith who brings stories to life with vivid descriptions and engaging narratives.', # Added backstory for the story writer
#     verbose=True,
#     llm=llm,
#     allow_delegation=False
# )

# content_formater=Agent(
#     role='Content Formatter',
#     goal='format the written story content in markdown',
#     backstory='a formatter who enchanced the readability and presentation of the storybook',
#     verbose=True,
#     llm=llm,
#     allow_delegation=False
# )

# #tasks
# task_outline=Task(
#     description='Create an outline for the children storybook about animals',
#     agent=story_outliner,
#     expected_output='A structured outline document containing 5 chater titles'
# )

# task_write=Task(
#     description='using the outline provided write the full story',
#     agent=story_writer,
#     expected_output='a complete manuscript'
# )

# task_format=Task(
#     description='Format the story content in markdown',
#     agent=content_formater,
#     expected_output='the entire storybook content formatted in markdown',
#     output_file='story.md'
# )

# crew=Crew(
#     agents=[story_outliner,story_writer,content_formater],
#     tasks=[task_outline,task_write,task_format],
#     verbose=True,
#     process=Process.sequential
# )


# result=crew.kickoff()
# print(result)

In [ ]:
from decimal import Context

user_interection=Agent(
    role="Interect with the customer to gain insights about the Supply Chain problem which the user is facing",
    goal="""The user would provide a transportation problem in which he has to transport a certain commodity from a starting point to a destination, he might list out some
    other neccesary information for better analysis like his approved budget, the max permisable delivery time, if there is any drop off location in between etc.

      Example 1
      User: XYZ Inc. has two factories with weekly production rates of 40 and 20 widgets. These widgets must be shipped to three warehouses that have a demand of 25, 10, and 25 units.
      The cost to ship between each location is known (see the grid below). Which factory should produce and ship how many widgets to which warehouses to meet the demand at each location
      with minimal cost?

      Supply at Factories:

      Factory A: 50 units
      Factory B: 60 units
      Factory C: 25 units

      Demand at Warehouses:

      Warehouse 1: 30 units
      Warehouse 2: 40 units
      Warehouse 3: 50 units
      Warehouse 4: 15 units

      Transportation Costs (per unit):

      From Factory A to:
      Warehouse 1: $8
      Warehouse 2: $6
      Warehouse 3: $10
      Warehouse 4: $9

      From Factory B to:
      Warehouse 1: $9
      Warehouse 2: $12
      Warehouse 3: $13
      Warehouse 4: $7

      From Factory C to:
      Warehouse 1: $14
      Warehouse 2: $9
      Warehouse 3: $16
      Warehouse 4: $5


      bot: Decision variables:
        Let x_ij be the quantity of widgets shipped from factory i to warehouse j.

          Constraints:
            Supply constraints:
            x_A1+x_A2+x_A3≤50
            x_B1+x_B2+x_B3≤60
            x_C1+x_C2+x_C3≤25

            Demand constraints:
            x_A1+x_B1+x_C1=30
            x_A2+x_B2+x_C2=40
            x_A3+x_B3+x_C3=50

            Non-negativity constraints:
            x_ij≥0 for all i,j""",
    backstory="A professional operations researcher specializing in transportation and logistics optimization",
    vervose=True,
    llm=llm
)

# Agent for formulating the transportation optimization problem
problem_formulate = Agent(
    role="Transportation Problem Formulator",
    goal="Given a transportation optimization scenario, formulate a linear programming model that minimizes travel time and cost while respecting a budget constraint.",
    backstory="A professional operations researcher specializing in transportation and logistics optimization",
    verbose=True,
    llm=llm
)

# Agent for validating and solving the formulated LP model
resoning = Agent(
    role='Transportation Problem Solver',
    goal='Review the formulated problem, validate the model, and provide a detailed solution using linear programming techniques to determine the optimal route and schedule.',
    backstory="A seasoned optimization expert with extensive experience in linear programming and supply chain management",
    verbose=True,
    llm=llm
)



In [ ]:
user=Task(
    description=""""Given a query of logistics management from the user your primary task is to output the supply and demand contraints which has to be then used by the next
    agent for the task of optimization. You are skilled in analyzing text and extract meaningful information. The output of this information has to be passed onto the next agent. """,
    expected_output=""" Output the supply and demand constraint based on the user query""",
    agent=user_interection,
    output_file='supply_demand.txt'
)


formulator = Task(
    description='''
    In a transportation network, there are multiple routes connecting various destinations.
    For example, assume you need to transport a product from Destination 1 to Destination 2.
    Each possible route has an associated travel time and cost.

    Your task is to formulate a linear programming model that:
    - Minimizes the total travel time and cost,
    - Ensures that the total transportation cost does not exceed a specified budget,
    - Considers the possibility of alternative routes and modes of transport.

    Clearly define the decision variables, objective function, and all necessary constraints.
    ''',
    expected_output='Formulated linear programming model for transportation optimization',
    agent=problem_formulate,
)

# Task for solving the formulated LP model
write = Task(
    description='''
    Solve the formulated transportation optimization problem using linear programming.
    Provide a detailed, step-by-step explanation that includes:
    - Definition of decision variables (e.g., whether a route is chosen or not),
    - Construction of the objective function that minimizes both travel time and cost,
    - Detailed constraints such as budget limits and route connectivity,
    - Explanation of the optimization process and the final solution,
    - How the solution achieves a balance between reducing travel time and staying within the budget.
    ''',
    expected_output='Detailed solution of the transportation optimization problem and give the final answer which the consumer should use for his transportation problem.',
    agent=resoning,
    output_file='solution.txt'
)

# Create the crew with both agents and tasks, and run the process sequentially
crew = Crew(
    agents=[user_interection,problem_formulate, resoning],
    tasks=[user,formulator, write],
    verbose=True,
    process=Process.sequential
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3ff6552a-4e87-4eae-9c48-05e4142c5eb3                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b726989c-a150-4f2c-bee7-20673b3c76b4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b726989c-a150-4f2c-bee7-20673b3c76b4
       Status: Executing Task...
    └── 🤖 Agent: Interect with the customer to gain insights about the Supply Chain problem which the user is 
        facing
            Status: In Progress

# Agent: Interect with the customer to gain insights about the Supply Chain problem which the user is facing
## Task: "Given a query of logistics management from the user your primary task is to output the supply and demand contraints which has to be then used by the next 
    agent for the task of optimization. You are skilled in analyzing text and extract meaningful information. The output of this information has to be passed onto the next agent. 


🚀 Crew: crew
└── 📋 Task: b726989c-a150-4f2c-bee7-20673b3c76b4
       Status: Executing Task...
    └── 🤖 Agent: Interect with the customer to gain insights about the Supply Chain problem which the user is 
        facing
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: b726989c-a150-4f2c-bee7-20673b3c76b4
│      Assigned to: Interect with the customer to gain insights about the Supply Chain problem which the user is 
│   facing
│      Status: ✅ Completed
│   └── 🤖 Agent: Interect with the customer to gain insights about the Supply Chain problem which the user is 
│       facing
│           Status: ✅ Completed
├── 📋 Task: 2f807cd5-a17a-4b37-9b7f-03844873dba4
│      Assigned to: Transportation Problem Formulator
│      Status: ✅ Completed
│   └── 🤖 Agent: Transportation Problem Formulator
│           Status: ✅ Completed
└── 📋 Task: 8639eab4-45b9-41f1-9117-781a320e3dad
       Status: Executing Task...
    └── 🤖 Agent: Transportation Problem Solver
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b726989c-a150-4f2c-bee7-20673b3c76b4
│      Assigned to: Interect with the customer to gain insights about the Supply Chain problem which the user is 
│   facing
│      Status: ✅ Completed
│   └── 🤖 Agent: Interect with the customer to gain insights about the Supply Chain problem which the user is 
│       facing
│           Status: ✅ Completed
├── 📋 Task: 2f807cd5-a17a-4b37-9b7f-03844873dba4
│      Assigned to: Transportation Problem Formulator
│      Status: ✅ Completed
│   └── 🤖 Agent: Transportation Problem Formulator
│           Status: ✅ Completed
└── 📋 Task: 8639eab4-45b9-41f1-9117-781a320e3dad
       Assigned to: Transportation Problem Solver
       Status: ✅ Completed
    └── 🤖 Agent: Transportation Problem Solver
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8639eab4-45b9-41f1-9117-781a320e3dad                                                                     │
│  Agent: Transportation Problem Solver                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3ff6552a-4e87-4eae-9c48-05e4142c5eb3                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

To provide the optimal solution and specific routes, I need the values for  `t_ij` (travel times) and `c_ij` (transportation costs) for each factory-destination pair.  

Once you provide those values, I can plug them into the linear programming model and use a solver (such as CPLEX, Gurobi, or even Excel Solver) to find the optimal solution. The solution will give you:

* **Optimal Shipping Quantities (x_ij):**  The amount to ship from each factory to each destination to minimize the total cost and time.
* **Minimum Total Cost and Time (Z):** The lowest combined cost and time achieved by the optimal solution.
* **Route Structure:** An indication of which routes are used to fulfill the demand based on the optimal shipping quantities.

**Here's a general outline of how the solution process works:**

1. **Data Input:** I'll need the `t_ij` and `c_ij` values for all factory-destination pairs.

2. **Model Formulation:** The linear programming model you provided will be used, with the specif

In [ ]:
from decimal import Context

# Assume necessary modules and classes (Agent, Task, Crew, Process) have been imported or defined

# Get a custom task description from the user
custom_task_input = input("Enter your transportation optimization problem description: ")

# Agent for formulating the transportation optimization problem
problem_formulate = Agent(
    role="Transportation Problem Formulator",
    goal="Given a transportation optimization scenario, formulate a linear programming model that minimizes travel time and cost while respecting a budget constraint.",
    backstory="A professional operations researcher specializing in transportation and logistics optimization",
    verbose=True,
    llm=llm
)

# Agent for validating and solving the formulated LP model
resoning = Agent(
    role='Transportation Problem Solver',
    goal='Review the formulated problem, validate the model, and provide a detailed solution using linear programming techniques to determine the optimal route and schedule.',
    backstory="A seasoned optimization expert with extensive experience in linear programming and supply chain management",
    verbose=True,
    llm=llm
)

# Create a custom Task using the user's input as the problem description
custom_task = Task(
    description=custom_task_input,
    expected_output='Detailed solution for the custom transportation optimization problem with step by step explanation and give the final answer also',
    agent=resoning,
    output_file='custom_solution.txt'
)

# Create the crew with both agents and the custom task, then run the process sequentially
crew = Crew(
    agents=[problem_formulate, resoning],
    tasks=[custom_task],
    verbose=True,
    process=Process.sequential
)

result = crew.kickoff()
print(result)


Enter your transportation optimization problem description: Niki holds two part-time jobs, Job I and Job II. She never wants to work more than a total of 12 hours a week. She has determined that for every hour she works at Job I, she needs 2 hours of preparation time, and for every hour she works at Job II, she needs one hour of preparation time, and she cannot spend more than 16 hours for preparation. If she makes $40 an hour at Job I, and $30 an hour at Job II, how many hours should she work per week at each job to maximize her income?


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d0a2e986-0820-46f5-a569-8c030bd61b6f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1e38c7cc-2e54-48f7-8132-208569e04d27
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1e38c7cc-2e54-48f7-8132-208569e04d27
       Status: Executing Task...
    └── 🤖 Agent: Transportation Problem Solver
            Status: In Progress

# Agent: Transportation Problem Solver
## Task: Niki holds two part-time jobs, Job I and Job II. She never wants to work more than a total of 12 hours a week. She has determined that for every hour she works at Job I, she needs 2 hours of preparation time, and for every hour she works at Job II, she needs one hour of preparation time, and she cannot spend more than 16 hours for preparation. If she makes $40 an hour at Job I, and $30 an hour at Job II, how many hours should she work per week at each job to maximize her income?


...

🚀 Crew: crew
└── 📋 Task: 1e38c7cc-2e54-48f7-8132-208569e04d27
       Status: Executing Task...
    └── 🤖 Agent: Transportation Problem Solver
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 1e38c7cc-2e54-48f7-8132-208569e04d27
       Assigned to: Transportation Problem Solver
       Status: ✅ Completed
    └── 🤖 Agent: Transportation Problem Solver
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1e38c7cc-2e54-48f7-8132-208569e04d27                                                                     │
│  Agent: Transportation Problem Solver                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d0a2e986-0820-46f5-a569-8c030bd61b6f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Let's formulate this problem as a linear programming model to determine the optimal number of hours Niki should work at each job.

**1. Define Decision Variables:**

*  Let  *x* represent the number of hours Niki works at Job I per week.
*  Let *y* represent the number of hours Niki works at Job II per week.

**2. Formulate the Objective Function:**

* Niki's income is given by $40 per hour at Job I and $30 per hour at Job II. Therefore, the objective function to maximize her income is: 

   Maximize Z = 40x + 30y

**3. State the Constraints:**

* **Time Constraint:** Niki cannot work more than 12 hours total per week:
    x + y ≤ 12

* **Preparation Time Constraint:**  
    2x + y ≤ 16

* **Non-negativity Constraints:**  Niki cannot work negative hours:
    x ≥ 0
    y ≥ 0

**4. Solve Using Linear Programming Techniques:**

To solve this problem, we can use graphical methods or software like Excel Solver or Python libraries (SciPy). Here's a step-by-step graphical solution:

   * **Pl

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Define an agent class for the transportation optimization problem
class TransportationProblemAgent:
    def __init__(self):
        # Define the data for the routes
        self.routes = ['Route1', 'Route2', 'Route3']
        # Travel times (in hours) for each route
        self.travel_time = {'Route1': 5, 'Route2': 3, 'Route3': 6}
        # Transportation costs (in dollars) for each route
        self.cost = {'Route1': 100, 'Route2': 150, 'Route3': 80}
        # Maximum allowed budget for transportation
        self.budget = 120

    def formulate_problem(self):
        # Formulate a description of the linear programming model
        formulation = f"""
        Transportation Optimization Model:
        -------------------------------------
        Decision Variables:
          For each route i ∈ {self.routes}, let x_i be a binary variable indicating whether route i is selected.

        Objective:
          Minimize total travel time:
              Minimize Z = ∑ (travel_time[i] * x_i)

        Constraints:
          1. Exactly one route must be selected:
              ∑ x_i = 1
          2. Budget constraint on transportation cost:
              ∑ (cost[i] * x_i) ≤ {self.budget}
        """
        return formulation

    def solve_problem(self):
        # Create a linear programming problem instance with PuLP
        prob = LpProblem("Transportation_Optimization", LpMinimize)

        # Decision variables: binary variables for each route
        x = LpVariable.dicts('x', self.routes, cat='Binary')

        # Objective: minimize total travel time
        prob += lpSum([self.travel_time[i] * x[i] for i in self.routes]), "Total_Travel_Time"

        # Constraint: Only one route must be selected
        prob += lpSum([x[i] for i in self.routes]) == 1, "One_Route_Selected"

        # Constraint: Total cost of the selected route must be within budget
        prob += lpSum([self.cost[i] * x[i] for i in self.routes]) <= self.budget, "Budget_Constraint"

        # Solve the LP problem
        prob.solve()

        # Collect the results
        result = f"Status: {LpStatus[prob.status]}\n"
        for r in self.routes:
            if value(x[r]) == 1:
                result += f"Selected route: {r}\n"
                result += f"Travel Time: {self.travel_time[r]} hours\n"
                result += f"Cost: ${self.cost[r]}\n"
        return result

# Create an instance of the agent
agent = TransportationProblemAgent()

# Formulate the problem and print the formulation
print("Formulated Problem:")
print(agent.formulate_problem())

# Solve the problem and print the solution
print("Solution:")
print(agent.solve_problem())


Formulated Problem:

        Transportation Optimization Model:
        -------------------------------------
        Decision Variables:
          For each route i ∈ ['Route1', 'Route2', 'Route3'], let x_i be a binary variable indicating whether route i is selected.
        
        Objective:
          Minimize total travel time: 
              Minimize Z = ∑ (travel_time[i] * x_i)
        
        Constraints:
          1. Exactly one route must be selected:
              ∑ x_i = 1
          2. Budget constraint on transportation cost:
              ∑ (cost[i] * x_i) ≤ 120
        
Solution:
Status: Optimal
Selected route: Route1
Travel Time: 5 hours
Cost: $100



In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 50.5 MB/s eta 0:00:00


In [ ]:
from crewai_tools import WebsiteSearchTool,FileReadTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://err

In [ ]:
!pip install databricks-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.5/677.5 kB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install crewai_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.0/559.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 